# Pokemon RBY Bot Version 1 

In [8]:
from poke_env.environment import AbstractBattle
from poke_env.player.battle_order import BattleOrder
from poke_env.player.player import Player
from typing import Any, Awaitable, Dict, List, Optional, Union
from poke_env.environment import AbstractBattle
from poke_env.player.battle_order import BattleOrder

import random
import numpy as np

from poke_env.environment.move_category import MoveCategory
from poke_env.environment.status import Status
from poke_env.environment.pokemon import Pokemon
from poke_env.teambuilder.teambuilder_pokemon import TeambuilderPokemon
from poke_env.environment.move import Move

In [9]:

## Damage Formula Optimized
import math
# Precompute the dmg_list outside the function as it does not change
DAMAGE_LIST = [217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

def damage(level, crit, power, atk, dfs, atk_mod, dfs_mod, stab, type_mult): 
    """
    Function to calculate damage
    level - attacker pokemon level
    crit - True or False depending of if move is a critical hit
    power - base power of move
    atk - attack stat of pokemon based on move type (special/physical)
    dfs - defense stat of pokemon based on move type (special/physical)
    stab - 1.5 if stab multiplier, 1 otherwise
    type_mult - based on type charts can range from 0-4

    """


    # Precompute common values
    lvl_value = (2 + (4 * level) // 5) if crit else (2 + (2 * level) // 5)
    max_atk_dfs = max(atk, dfs)
    
    # Check if atk or dfs exceeds 255 and adjust values accordingly
    if max_atk_dfs > 255:
        # Scaling for high attack or defense values
        at = (atk // 4) if crit else ((atk_mod * atk) // 4)
        df = (dfs // 4) if crit else ((dfs_mod * dfs) // 4)
    else:
        # No scaling for normal values
        at = atk if crit else (atk_mod * atk)
        df = dfs if crit else (dfs_mod * dfs)

    # Calculate damage
    dmg_1 = ((lvl_value * power * at) // df) // 50 + 2
    final_dmg_value = math.floor(math.floor(dmg_1 * stab) * type_mult)

    # If final damage value is 1, return a list of 1s
    if final_dmg_value == 1:
        return [1] * 39

    # Otherwise, return a scaled list of damage values
    return [(final_dmg_value * x) // 255 for x in DAMAGE_LIST]


In [10]:
TypeChart = {
    '???': {
        'NORMAL': 1,
        'GRASS': 1,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 1,
        'ROCK': 1,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 1,
    },
    'NORMAL': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 1,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 1,
        'ROCK': 0.5,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 0,
        'DRAGON': 1,
    },
    'GRASS': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 0.5,
        'FIRE': 0.5,
        'WATER': 2,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 0.5,
        'BUG': 0.5,
        'POISON': 0.5,
        'GROUND': 2,
        'ROCK': 2,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 0.5,
    },
    'FIRE': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 2,
        'FIRE': 0.5,
        'WATER': 0.5,
        'ELECTRIC': 1,
        'ICE': 2,
        'FLYING': 1,
        'BUG': 2,
        'POISON': 1,
        'GROUND': 1,
        'ROCK': 0.5,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 0.5,
    },
    'WATER': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 0.5,
        'FIRE': 2,
        'WATER': 0.5,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 2,
        'ROCK': 2,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 0.5,
    },
    'ELECTRIC': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 0.5,
        'FIRE': 1,
        'WATER': 2,
        'ELECTRIC': 0.5,
        'ICE': 1,
        'FLYING': 2,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 0,
        'ROCK': 1,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 0.5,
    },
    'ICE': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 2,
        'FIRE': 1,
        'WATER': 0.5,
        'ELECTRIC': 1,
        'ICE': 0.5,
        'FLYING': 2,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 2,
        'ROCK': 1,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 2,
    },
    'FLYING': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 2,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 0.5,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 2,
        'POISON': 1,
        'GROUND': 1,
        'ROCK': 0.5,
        'FIGHTING': 2,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 1,
    },
    'BUG': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 2,
        'FIRE': 0.5,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 0.5,
        'BUG': 1,
        'POISON': 2,
        'GROUND': 1,
        'ROCK': 1,
        'FIGHTING': 0.5,
        'PSYCHIC': 2,
        'GHOST': 0.5,
        'DRAGON': 1,
    },
    'POISON': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 2,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 2,
        'POISON': 0.5,
        'GROUND': 0.5,
        'ROCK': 0.5,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 0.5,
        'DRAGON': 1,
    },
    'GROUND': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 0.5,
        'FIRE': 2,
        'WATER': 1,
        'ELECTRIC': 2,
        'ICE': 1,
        'FLYING': 0,
        'BUG': 0.5,
        'POISON': 2,
        'GROUND': 1,
        'ROCK': 2,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 1,
    },
    'ROCK': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 1,
        'FIRE': 2,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 2,
        'FLYING': 2,
        'BUG': 2,
        'POISON': 1,
        'GROUND': 0.5,
        'ROCK': 1,
        'FIGHTING': 0.5,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 1,
    },
    'FIGHTING': {
        '???': 1,
        'NORMAL': 2,
        'GRASS': 1,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 2,
        'FLYING': 0.5,
        'BUG': 0.5,
        'POISON': 0.5,
        'GROUND': 1,
        'ROCK': 2,
        'FIGHTING': 1,
        'PSYCHIC': 0.5,
        'GHOST': 0,
        'DRAGON': 1,
    },
    'PSYCHIC': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 1,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 1,
        'POISON': 2,
        'GROUND': 1,
        'ROCK': 1,
        'FIGHTING': 2,
        'PSYCHIC': 0.5,
        'GHOST': 1,
        'DRAGON': 1,
    },
    'GHOST': {
        '???': 1,
        'NORMAL': 0,
        'GRASS': 1,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 1,
        'ROCK': 1,
        'FIGHTING': 1,
        'PSYCHIC': 0,
        'GHOST': 2,
        'DRAGON': 1,
    },
    'DRAGON': {
        '???': 1,
        'NORMAL': 1,
        'GRASS': 1,
        'FIRE': 1,
        'WATER': 1,
        'ELECTRIC': 1,
        'ICE': 1,
        'FLYING': 1,
        'BUG': 1,
        'POISON': 1,
        'GROUND': 1,
        'ROCK': 1,
        'FIGHTING': 1,
        'PSYCHIC': 1,
        'GHOST': 1,
        'DRAGON': 2,
    }
}


In [11]:
MoveData = {
    '(No Move)': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Absorb': {'bp': 20, 'type': 'Grass', 'drain': [1, 2]},
    'Acid': {'bp': 40, 'type': 'Poison'},
    'Amnesia': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Aurora Beam': {'bp': 65, 'type': 'Ice'},
    'Barrage': {'bp': 15, 'type': 'Normal', 'multihit': [2, 5]},
    'Bide': {'bp': 0, 'type': '???'},
    'Bind': {'bp': 15, 'type': 'Normal'},
    'Bite': {'bp': 60, 'type': 'Normal'},
    'Blizzard': {'bp': 120, 'type': 'Ice'},
    'Bonemerang': {'bp': 50, 'type': 'Ground', 'multihit': 2},
    'Bubble': {'bp': 20, 'type': 'Water'},
    'Bubble Beam': {'bp': 65, 'type': 'Water'},
    'Clamp': {'bp': 35, 'type': 'Water'},
    'Comet Punch': {'bp': 18, 'type': 'Normal', 'multihit': [2, 5]},
    'Constrict': {'bp': 10, 'type': 'Normal'},
    'Conversion': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Counter': {'bp': 1, 'type': 'Fighting'},
    'Crabhammer': {'bp': 90, 'type': 'Water'},
    'Defense Curl': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Dig': {'bp': 100, 'type': 'Ground'},
    'Disable': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Dizzy Punch': {'bp': 70, 'type': 'Normal'},
    'Double-Edge': {'bp': 100, 'type': 'Normal', 'recoil': [25, 100]},
    'Double Kick': {'bp': 30, 'type': 'Fighting', 'multihit': 2},
    'Double Slap': {'bp': 15, 'type': 'Normal', 'multihit': [2, 5]},
    'Dragon Rage': {'bp': 1, 'type': 'Dragon'},
    'Dream Eater': {'bp': 100, 'type': 'Psychic', 'drain': [1, 2]},
    'Earthquake': {'bp': 100, 'type': 'Ground'},
    'Explosion': {'bp': 170, 'type': 'Normal'},
    'Fire Blast': {'bp': 120, 'type': 'Fire'},
    'Fire Spin': {'bp': 15, 'type': 'Fire'},
    'Fissure': {'bp': 0, 'type': 'Ground'},
    'Fly': {'bp': 70, 'type': 'Flying'},
    'Focus Energy': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Fury Attack': {'bp': 15, 'type': 'Normal', 'multihit': [2, 5]},
    'Fury Swipes': {'bp': 18, 'type': 'Normal', 'multihit': [2, 5]},
    'Glare': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Growth': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Guillotine': {'bp': 0, 'type': 'Normal'},
    'Gust': {'bp': 40, 'type': 'Normal'},
    'Haze': {'bp': 0, 'category': 'Status', 'type': 'Ice'},
    'High Jump Kick': {'bp': 85, 'type': 'Fighting', 'hasCrashDamage': True},
    'Horn Drill': {'bp': 0, 'type': 'Normal'},
    'Hyper Beam': {'bp': 150, 'type': 'Normal'},
    'Jump Kick': {'bp': 70, 'type': 'Fighting', 'hasCrashDamage': True},
    'Karate Chop': {'bp': 50, 'type': 'Normal'},
    'Leech Seed': {'bp': 0, 'category': 'Status', 'type': 'Grass'},
    'Light Screen': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Metronome': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Mimic': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Minimize': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Mirror Move': {'bp': 0, 'category': 'Status', 'type': 'Flying'},
    'Mist': {'bp': 0, 'category': 'Status', 'type': 'Ice'},
    'Night Shade': {'bp': 1, 'type': 'Ghost'},
    'Petal Dance': {'bp': 70, 'type': 'Grass'},
    'Pin Missile': {'bp': 14, 'type': 'Bug', 'multihit': [2, 5]},
    'Poison Sting': {'bp': 15, 'type': 'Poison'},
    'Psychic': {'bp': 90, 'type': 'Psychic'},
    'Psywave': {'bp': 1, 'type': 'Psychic'},
    'Rage': {'bp': 20, 'type': 'Normal'},
    'Razor Leaf': {'bp': 55, 'type': 'Grass'},
    'Razor Wind': {'bp': 80, 'type': 'Normal'},
    'Recover': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Reflect': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Rest': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Roar': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Rock Slide': {'bp': 75, 'type': 'Rock'},
    'Rock Throw': {'bp': 50, 'type': 'Rock'},
    'Sand Attack': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Seismic Toss': {'bp': 1, 'type': 'Fighting'},
    'Self-Destruct': {'bp': 130, 'type': 'Normal'},
    'Skull Bash': {'bp': 100, 'type': 'Normal'},
    'Slash': {'bp': 70, 'type': 'Normal'},
    'Sludge': {'bp': 65, 'type': 'Poison'},
    'Soft-Boiled': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Solar Beam': {'bp': 120, 'type': 'Grass'},
    'Sonic Boom': {'bp': 1, 'type': 'Normal'},
    'Spike Cannon': {'bp': 20, 'type': 'Normal', 'multihit': [2, 5]},
    'Stomp': {'bp': 65, 'type': 'Normal'},
    'Struggle': {'bp': 50, 'type': 'Normal', 'recoil': [1, 2]},
    'Stun Spore': {'bp': 0, 'category': 'Status', 'type': 'Grass'},
    'Submission': {'bp': 80, 'type': 'Fighting', 'recoil': [1, 4]},
    'Substitute': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Super Fang': {'bp': 1, 'type': 'Normal'},
    'Swift': {'bp': 60, 'type': 'Normal'},
    'Take Down': {'bp': 90, 'type': 'Normal', 'recoil': [1, 4]},
    'Thrash': {'bp': 90, 'type': 'Normal'},
    'Thunder': {'bp': 120, 'type': 'Electric'},
    'Thunder Wave': {'bp': 0, 'category': 'Status', 'type': 'Electric'},
    'Transform': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Tri Attack': {'bp': 80, 'type': 'Normal'},
    'Twineedle': {'bp': 25, 'type': 'Bug', 'multihit': 2},
    'Whirlwind': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Wing Attack': {'bp': 35, 'type': 'Flying'},
    'Wrap': {'bp': 15, 'type': 'Normal'},
    'Growl': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Leer': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Low Kick': {'bp': 50, 'type': 'Fighting'},
    'Poison Gas': {'bp': 0, 'category': 'Status', 'type': 'Poison'},
    'Poison Powder': {'bp': 0, 'category': 'Status', 'type': 'Poison'},
    'Sky Attack': {'bp': 140, 'type': 'Flying'},
    'String Shot': {'bp': 0, 'category': 'Status', 'type': 'Bug'},
    'Surf': {'bp': 95, 'type': 'Water'},
    'Tail Whip': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Toxic': {'bp': 0, 'category': 'Status', 'type': 'Poison'},
    'Flash': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Hypnosis': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Leech Life': {'bp': 20, 'type': 'Bug', 'drain': [1, 2]},
    'Mega Drain': {'bp': 40, 'type': 'Grass', 'drain': [1, 2]},
    'Vine Whip': {'bp': 35, 'type': 'Grass'},
    'Waterfall': {'bp': 80, 'type': 'Water'},
    'Tackle': {'bp': 35, 'type': 'Normal'},
    'Acid Armor': {'bp': 0, 'category': 'Status', 'type': 'Poison'},
    'Barrier': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Body Slam': {'bp': 85, 'type': 'Normal'},
    'Flamethrower': {'bp': 95, 'type': 'Fire'},
    'Hydro Pump': {'bp': 120, 'type': 'Water'},
    'Ice Beam': {'bp': 95, 'type': 'Ice'},
    'Lick': {'bp': 20, 'type': 'Ghost'},
    'Screech': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Sing': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Sleep Powder': {'bp': 0, 'category': 'Status', 'type': 'Grass'},
    'Smog': {'bp': 20, 'type': 'Poison'},
    'Spore': {'bp': 0, 'category': 'Status', 'type': 'Grass'},
    'Supersonic': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Swords Dance': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Thunderbolt': {'bp': 95, 'type': 'Electric'},
    'Bone Club': {'bp': 65, 'type': 'Ground'},
    'Egg Bomb': {'bp': 100, 'type': 'Normal'},
    'Hyper Fang': {'bp': 80, 'type': 'Normal'},
    'Kinesis': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Lovely Kiss': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Meditate': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Rolling Kick': {'bp': 60, 'type': 'Fighting'},
    'Sharpen': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Teleport': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Agility': {'bp': 0, 'category': 'Status', 'type': 'Psychic'},
    'Confuse Ray': {'bp': 0, 'category': 'Status', 'type': 'Ghost'},
    'Confusion': {'bp': 50, 'type': 'Psychic'},
    'Cut': {'bp': 50, 'type': 'Normal'},
    'Double Team': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Drill Peck': {'bp': 80, 'type': 'Flying'},
    'Ember': {'bp': 40, 'type': 'Fire'},
    'Fire Punch': {'bp': 75, 'type': 'Fire'},
    'Harden': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Headbutt': {'bp': 70, 'type': 'Normal'},
    'Horn Attack': {'bp': 65, 'type': 'Normal'},
    'Ice Punch': {'bp': 75, 'type': 'Ice'},
    'Mega Kick': {'bp': 120, 'type': 'Normal'},
    'Mega Punch': {'bp': 80, 'type': 'Normal'},
    'Paleo Wave': {'bp': 85, 'type': 'Rock'},
    'Pay Day': {'bp': 40, 'type': 'Normal'},
    'Peck': {'bp': 35, 'type': 'Flying'},
    'Pound': {'bp': 40, 'type': 'Normal'},
    'Psybeam': {'bp': 65, 'type': 'Psychic'},
    'Quick Attack': {'bp': 40, 'type': 'Normal', 'priority': 1},
    'Scratch': {'bp': 40, 'type': 'Normal'},
    'Shadow Strike': {'bp': 80, 'type': 'Ghost'},
    'Slam': {'bp': 80, 'type': 'Normal'},
    'Smokescreen': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Splash': {'bp': 0, 'category': 'Status', 'type': 'Normal'},
    'Strength': {'bp': 80, 'type': 'Normal'},
    'Thunder Punch': {'bp': 75, 'type': 'Electric'},
    'Thunder Shock': {'bp': 40, 'type': 'Electric'},
    'Vise Grip': {'bp': 55, 'type': 'Normal'},
    'Water Gun': {'bp': 40, 'type': 'Water'},
    'Withdraw': {'bp': 0, 'category': 'Status', 'type': 'Water'}
}


In [12]:
move_list = list(MoveData.keys())
move_list = [x.lower() for x in move_list]
move_list = [x.replace(' ','') for x in move_list]
move_list = [x.replace('-','') for x in move_list]

In [13]:
move_list.pop(0)

'(nomove)'

In [14]:
move_dict = {key:Move(key,1) for key in move_list}

In [10]:
import pandas as pd
stats = pd.read_csv('pokemon-bst-totals.csv')


In [11]:

stats['Name'] = stats['Name'].apply(str.lower)

In [12]:
stats.iloc[26:35]

,Name,HP,Attack,Defense,Speed,Special,Total,HP_Total,Attack_Total,Defense_Total,Speed_Total,Special_Total
26,sandshrew,50,75,85,40,30,280,303,248,268,178,158
27,sandslash,75,100,110,65,55,405,353,298,318,228,208
28,nidoranf,55,47,52,41,40,235,313,192,202,180,178
29,nidorina,70,62,67,56,55,310,343,222,232,210,208
30,nidoqueen,90,82,87,76,75,410,383,262,272,250,248
31,nidoranm,46,57,40,50,40,233,295,212,178,198,178
32,nidorino,61,72,57,65,55,310,325,242,212,228,208
33,nidoking,81,92,77,85,75,410,365,282,252,268,248
34,clefairy,70,45,48,35,60,258,343,188,194,168,218


In [13]:
stats.set_index('Name',inplace=True)

In [14]:
import json 
f = open('gen1moves.json')
FullMoveData = json.load(f)

In [15]:
FullMoveData['thunderwave']

{'accuracy': 100,
 'basePower': 0,
 'category': 'Status',
 'contestType': 'Cool',
 'flags': {'metronome': 1, 'mirror': 1, 'protect': 1, 'reflectable': 1},
 'ignoreImmunity': False,
 'name': 'Thunder Wave',
 'num': 86,
 'pp': 20,
 'priority': 0,
 'secondary': None,
 'status': 'par',
 'target': 'normal',
 'type': 'Electric',
 'zMove': {'boost': {'spd': 1}}}

In [16]:
#dict of all possible moves for each pokemon
pkm_mvs_dict = {'bulbasaur': ['bide',
  'tackle',
  'razor-leaf',
  'reflect',
  'mega-drain',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'sleep-powder',
  'rest',
  'growth',
  'rage',
  'toxic',
  'leech-seed',
  'take-down',
  'growl',
  'double-team',
  'double-edge',
  'cut',
  'body-slam',
  'poison-powder',
  'mimic'],
 'ivysaur': ['bide',
  'tackle',
  'razor-leaf',
  'reflect',
  'mega-drain',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'rest',
  'sleep-powder',
  'growth',
  'leech-seed',
  'toxic',
  'rage',
  'take-down',
  'growl',
  'double-team',
  'double-edge',
  'cut',
  'body-slam',
  'poison-powder',
  'mimic'],
 'venusaur': ['hyper-beam',
  'bide',
  'tackle',
  'razor-leaf',
  'reflect',
  'mega-drain',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'rest',
  'sleep-powder',
  'growth',
  'leech-seed',
  'toxic',
  'rage',
  'take-down',
  'growl',
  'double-team',
  'double-edge',
  'cut',
  'body-slam',
  'poison-powder',
  'mimic'],
 'charmander': ['dragon-rage',
  'mega-kick',
  'bide',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'substitute',
  'scratch',
  'swords-dance',
  'fire-blast',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'slash',
  'take-down',
  'strength',
  'leer',
  'growl',
  'double-team',
  'mega-punch',
  'double-edge',
  'cut',
  'swift',
  'body-slam',
  'fire-spin',
  'mimic',
  'seismic-toss'],
 'charmeleon': ['dragon-rage',
  'bide',
  'mega-kick',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'substitute',
  'scratch',
  'swords-dance',
  'fire-blast',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'slash',
  'take-down',
  'strength',
  'leer',
  'growl',
  'double-team',
  'mega-punch',
  'double-edge',
  'cut',
  'swift',
  'body-slam',
  'fire-spin',
  'mimic',
  'seismic-toss'],
 'charizard': ['dragon-rage',
  'hyper-beam',
  'bide',
  'mega-kick',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'substitute',
  'scratch',
  'swords-dance',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'slash',
  'take-down',
  'strength',
  'leer',
  'growl',
  'double-team',
  'mega-punch',
  'double-edge',
  'cut',
  'swift',
  'body-slam',
  'fire-spin',
  'mimic',
  'seismic-toss'],
 'squirtle': ['mega-kick',
  'bide',
  'tackle',
  'reflect',
  'skull-bash',
  'substitute',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'double-team',
  'bubble',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'withdraw',
  'seismic-toss'],
 'wartortle': ['bide',
  'mega-kick',
  'tackle',
  'reflect',
  'skull-bash',
  'substitute',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'double-team',
  'bubble',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'withdraw',
  'seismic-toss'],
 'blastoise': ['hyper-beam',
  'bide',
  'mega-kick',
  'tackle',
  'reflect',
  'skull-bash',
  'substitute',
  'earthquake',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'double-team',
  'bubble',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'withdraw',
  'seismic-toss'],
 'caterpie': ['tackle', 'string-shot'],
 'metapod': ['harden', 'tackle', 'string-shot'],
 'butterfree': ['string-shot',
  'hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'psybeam',
  'reflect',
  'psychic',
  'mega-drain',
  'substitute',
  'whirlwind',
  'solar-beam',
  'flash',
  'sleep-powder',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'supersonic',
  'double-team',
  'harden',
  'double-edge',
  'swift',
  'gust',
  'psywave',
  'poison-powder',
  'stun-spore',
  'confusion',
  'mimic',
  'razor-wind'],
 'weedle': ['poison-sting', 'string-shot'],
 'kakuna': ['harden', 'poison-sting', 'string-shot'],
 'beedrill': ['string-shot',
  'hyper-beam',
  'bide',
  'reflect',
  'skull-bash',
  'mega-drain',
  'agility',
  'substitute',
  'swords-dance',
  'fury-attack',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'harden',
  'double-edge',
  'cut',
  'swift',
  'twineedle',
  'mimic',
  'pin-missile',
  'focus-energy'],
 'pidgey': ['wing-attack',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'whirlwind',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'sand-attack',
  'take-down',
  'double-team',
  'double-edge',
  'swift',
  'gust',
  'quick-attack',
  'mimic',
  'mirror-move',
  'razor-wind'],
 'pidgeotto': ['wing-attack',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'whirlwind',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'sand-attack',
  'take-down',
  'double-team',
  'double-edge',
  'swift',
  'gust',
  'quick-attack',
  'mimic',
  'mirror-move',
  'razor-wind'],
 'pidgeot': ['wing-attack',
  'hyper-beam',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'whirlwind',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'sand-attack',
  'take-down',
  'double-team',
  'double-edge',
  'swift',
  'gust',
  'quick-attack',
  'mimic',
  'mirror-move',
  'razor-wind'],
 'rattata': ['thunderbolt',
  'hyper-fang',
  'bide',
  'tackle',
  'skull-bash',
  'substitute',
  'rest',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'tail-whip',
  'double-team',
  'super-fang',
  'double-edge',
  'swift',
  'body-slam',
  'quick-attack',
  'mimic',
  'thunder',
  'focus-energy'],
 'raticate': ['thunderbolt',
  'hyper-beam',
  'hyper-fang',
  'bide',
  'tackle',
  'skull-bash',
  'substitute',
  'rest',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'tail-whip',
  'double-team',
  'super-fang',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'quick-attack',
  'mimic',
  'thunder',
  'focus-energy'],
 'spearow': ['bide',
  'agility',
  'substitute',
  'whirlwind',
  'fury-attack',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'take-down',
  'drill-peck',
  'leer',
  'growl',
  'double-team',
  'double-edge',
  'swift',
  'peck',
  'mimic',
  'mirror-move',
  'razor-wind'],
 'fearow': ['hyper-beam',
  'bide',
  'substitute',
  'agility',
  'whirlwind',
  'fury-attack',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'take-down',
  'drill-peck',
  'leer',
  'growl',
  'double-team',
  'double-edge',
  'swift',
  'peck',
  'mimic',
  'mirror-move',
  'razor-wind'],
 'ekans': ['glare',
  'bide',
  'acid',
  'rock-slide',
  'screech',
  'skull-bash',
  'mega-drain',
  'substitute',
  'earthquake',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'wrap',
  'double-edge',
  'bite',
  'body-slam',
  'mimic'],
 'arbok': ['glare',
  'hyper-beam',
  'bide',
  'acid',
  'rock-slide',
  'screech',
  'skull-bash',
  'mega-drain',
  'substitute',
  'earthquake',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'wrap',
  'double-edge',
  'bite',
  'body-slam',
  'mimic'],
 'pikachu': ['thunderbolt',
  'mega-kick',
  'bide',
  'reflect',
  'skull-bash',
  'slam',
  'agility',
  'substitute',
  'flash',
  'rest',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'submission',
  'thunder-shock',
  'take-down',
  'tail-whip',
  'growl',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'quick-attack',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder'],
 'raichu': ['thunderbolt',
  'hyper-beam',
  'mega-kick',
  'bide',
  'reflect',
  'skull-bash',
  'slam',
  'agility',
  'substitute',
  'flash',
  'rest',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'submission',
  'thunder-shock',
  'take-down',
  'tail-whip',
  'growl',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'quick-attack',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder'],
 'sandshrew': ['bide',
  'rock-slide',
  'skull-bash',
  'substitute',
  'scratch',
  'swords-dance',
  'earthquake',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'submission',
  'slash',
  'sand-attack',
  'take-down',
  'strength',
  'double-team',
  'fury-swipes',
  'double-edge',
  'cut',
  'swift',
  'body-slam',
  'mimic',
  'seismic-toss'],
 'sandslash': ['hyper-beam',
  'bide',
  'rock-slide',
  'skull-bash',
  'substitute',
  'scratch',
  'swords-dance',
  'earthquake',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'submission',
  'slash',
  'sand-attack',
  'take-down',
  'strength',
  'double-team',
  'fury-swipes',
  'double-edge',
  'cut',
  'swift',
  'body-slam',
  'mimic',
  'seismic-toss'],
 'nidoran-f': ['thunderbolt',
  'bide',
  'tackle',
  'reflect',
  'double-kick',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'blizzard',
  'take-down',
  'tail-whip',
  'growl',
  'double-team',
  'fury-swipes',
  'double-edge',
  'bite',
  'body-slam',
  'mimic',
  'thunder'],
 'nidorina': ['thunderbolt',
  'bide',
  'tackle',
  'reflect',
  'double-kick',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'tail-whip',
  'growl',
  'double-team',
  'horn-drill',
  'fury-swipes',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'thunder'],
 'nidoqueen': ['thunderbolt',
  'hyper-beam',
  'bide',
  'mega-kick',
  'tackle',
  'reflect',
  'double-kick',
  'rock-slide',
  'skull-bash',
  'substitute',
  'scratch',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'poison-sting',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'growl',
  'double-team',
  'horn-drill',
  'fury-swipes',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder'],
 'nidoran-m': ['thunderbolt',
  'bide',
  'tackle',
  'reflect',
  'double-kick',
  'horn-attack',
  'skull-bash',
  'substitute',
  'fury-attack',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'horn-drill',
  'double-edge',
  'body-slam',
  'mimic',
  'thunder',
  'focus-energy'],
 'nidorino': ['thunderbolt',
  'bide',
  'tackle',
  'reflect',
  'double-kick',
  'horn-attack',
  'skull-bash',
  'substitute',
  'fury-attack',
  'rest',
  'poison-sting',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'horn-drill',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'thunder',
  'focus-energy'],
 'nidoking': ['thrash',
  'thunderbolt',
  'hyper-beam',
  'bide',
  'mega-kick',
  'tackle',
  'reflect',
  'double-kick',
  'rock-slide',
  'horn-attack',
  'skull-bash',
  'substitute',
  'fire-blast',
  'fury-attack',
  'earthquake',
  'rest',
  'counter',
  'poison-sting',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'horn-drill',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder',
  'focus-energy'],
 'clefairy': ['thunderbolt',
  'tri-attack',
  'mega-kick',
  'teleport',
  'bide',
  'sing',
  'reflect',
  'double-slap',
  'psychic',
  'skull-bash',
  'substitute',
  'solar-beam',
  'fire-blast',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'pound',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'defense-curl',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder',
  'metronome'],
 'clefable': ['thunderbolt',
  'tri-attack',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'sing',
  'reflect',
  'double-slap',
  'psychic',
  'skull-bash',
  'substitute',
  'solar-beam',
  'fire-blast',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'pound',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'defense-curl',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder',
  'metronome'],
 'vulpix': ['confuse-ray',
  'bide',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'roar',
  'take-down',
  'tail-whip',
  'double-team',
  'double-edge',
  'swift',
  'body-slam',
  'fire-spin',
  'quick-attack',
  'mimic'],
 'ninetales': ['confuse-ray',
  'hyper-beam',
  'bide',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'roar',
  'take-down',
  'tail-whip',
  'double-team',
  'double-edge',
  'swift',
  'body-slam',
  'fire-spin',
  'quick-attack',
  'mimic'],
 'jigglypuff': ['thunderbolt',
  'tri-attack',
  'mega-kick',
  'teleport',
  'bide',
  'sing',
  'disable',
  'reflect',
  'double-slap',
  'psychic',
  'skull-bash',
  'substitute',
  'solar-beam',
  'fire-blast',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'pound',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'double-team',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'defense-curl',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder'],
 'wigglytuff': ['thunderbolt',
  'tri-attack',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'sing',
  'disable',
  'reflect',
  'double-slap',
  'psychic',
  'skull-bash',
  'substitute',
  'solar-beam',
  'fire-blast',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'pound',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'double-team',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'defense-curl',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder'],
 'zubat': ['haze',
  'wing-attack',
  'confuse-ray',
  'bide',
  'mega-drain',
  'substitute',
  'whirlwind',
  'rest',
  'rage',
  'toxic',
  'leech-life',
  'take-down',
  'supersonic',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'mimic',
  'razor-wind'],
 'golbat': ['haze',
  'wing-attack',
  'confuse-ray',
  'hyper-beam',
  'bide',
  'screech',
  'mega-drain',
  'substitute',
  'whirlwind',
  'rest',
  'rage',
  'toxic',
  'leech-life',
  'take-down',
  'supersonic',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'mimic',
  'razor-wind'],
 'oddish': ['absorb',
  'bide',
  'acid',
  'reflect',
  'mega-drain',
  'substitute',
  'swords-dance',
  'solar-beam',
  'rest',
  'sleep-powder',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'double-edge',
  'cut',
  'poison-powder',
  'stun-spore',
  'mimic',
  'petal-dance'],
 'gloom': ['absorb',
  'bide',
  'acid',
  'reflect',
  'mega-drain',
  'substitute',
  'swords-dance',
  'solar-beam',
  'sleep-powder',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'double-edge',
  'cut',
  'poison-powder',
  'stun-spore',
  'mimic',
  'petal-dance'],
 'vileplume': ['absorb',
  'hyper-beam',
  'bide',
  'acid',
  'reflect',
  'mega-drain',
  'substitute',
  'swords-dance',
  'solar-beam',
  'sleep-powder',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'double-edge',
  'cut',
  'body-slam',
  'poison-powder',
  'stun-spore',
  'mimic',
  'petal-dance'],
 'paras': ['spore',
  'bide',
  'reflect',
  'skull-bash',
  'mega-drain',
  'substitute',
  'scratch',
  'swords-dance',
  'solar-beam',
  'rest',
  'growth',
  'rage',
  'toxic',
  'leech-life',
  'slash',
  'take-down',
  'double-team',
  'double-edge',
  'cut',
  'body-slam',
  'stun-spore',
  'mimic'],
 'parasect': ['spore',
  'hyper-beam',
  'bide',
  'reflect',
  'skull-bash',
  'mega-drain',
  'substitute',
  'scratch',
  'swords-dance',
  'solar-beam',
  'rest',
  'growth',
  'rage',
  'toxic',
  'leech-life',
  'slash',
  'take-down',
  'double-team',
  'double-edge',
  'cut',
  'body-slam',
  'stun-spore',
  'mimic'],
 'venonat': ['bide',
  'tackle',
  'disable',
  'reflect',
  'psybeam',
  'psychic',
  'mega-drain',
  'substitute',
  'solar-beam',
  'flash',
  'sleep-powder',
  'rest',
  'rage',
  'toxic',
  'leech-life',
  'take-down',
  'supersonic',
  'double-team',
  'double-edge',
  'psywave',
  'poison-powder',
  'stun-spore',
  'confusion',
  'mimic'],
 'venomoth': ['hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'disable',
  'reflect',
  'psybeam',
  'psychic',
  'mega-drain',
  'substitute',
  'whirlwind',
  'solar-beam',
  'flash',
  'rest',
  'sleep-powder',
  'rage',
  'toxic',
  'leech-life',
  'take-down',
  'supersonic',
  'double-team',
  'double-edge',
  'swift',
  'psywave',
  'poison-powder',
  'stun-spore',
  'confusion',
  'mimic',
  'razor-wind'],
 'diglett': ['toxic',
  'double-edge',
  'cut',
  'substitute',
  'scratch',
  'body-slam',
  'slash',
  'take-down',
  'sand-attack',
  'earthquake',
  'bide',
  'mimic',
  'rest',
  'growl',
  'double-team',
  'rock-slide',
  'rage'],
 'dugtrio': ['toxic',
  'double-edge',
  'substitute',
  'cut',
  'scratch',
  'body-slam',
  'slash',
  'hyper-beam',
  'take-down',
  'sand-attack',
  'earthquake',
  'bide',
  'mimic',
  'rest',
  'growl',
  'double-team',
  'rock-slide',
  'rage'],
 'meowth': ['thunderbolt',
  'bide',
  'screech',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'pay-day',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'slash',
  'take-down',
  'growl',
  'double-team',
  'fury-swipes',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'mimic',
  'thunder'],
 'persian': ['thunderbolt',
  'hyper-beam',
  'bide',
  'screech',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'pay-day',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'slash',
  'take-down',
  'growl',
  'double-team',
  'fury-swipes',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'mimic',
  'thunder'],
 'psyduck': ['mega-kick',
  'bide',
  'disable',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'counter',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'double-team',
  'fury-swipes',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'confusion',
  'mimic',
  'seismic-toss'],
 'golduck': ['hyper-beam',
  'bide',
  'mega-kick',
  'disable',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'counter',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'double-team',
  'fury-swipes',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'confusion',
  'mimic',
  'seismic-toss'],
 'mankey': ['thrash',
  'thunderbolt',
  'mega-kick',
  'bide',
  'rock-slide',
  'screech',
  'skull-bash',
  'substitute',
  'scratch',
  'metronome',
  'rest',
  'counter',
  'pay-day',
  'rage',
  'toxic',
  'submission',
  'karate-chop',
  'take-down',
  'strength',
  'leer',
  'low-kick',
  'double-team',
  'fury-swipes',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder',
  'focus-energy'],
 'primeape': ['thrash',
  'thunderbolt',
  'hyper-beam',
  'mega-kick',
  'bide',
  'rock-slide',
  'screech',
  'skull-bash',
  'substitute',
  'scratch',
  'rest',
  'counter',
  'pay-day',
  'rage',
  'toxic',
  'submission',
  'focus-energy',
  'karate-chop',
  'take-down',
  'strength',
  'leer',
  'low-kick',
  'double-team',
  'fury-swipes',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder',
  'metronome'],
 'growlithe': ['dragon-rage',
  'bide',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'roar',
  'take-down',
  'leer',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'mimic'],
 'arcanine': ['dragon-rage',
  'hyper-beam',
  'bide',
  'teleport',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'roar',
  'take-down',
  'leer',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'mimic'],
 'poliwag': ['bide',
  'double-slap',
  'amnesia',
  'psychic',
  'skull-bash',
  'substitute',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'hypnosis',
  'blizzard',
  'take-down',
  'double-team',
  'bubble',
  'double-edge',
  'ice-beam',
  'body-slam',
  'psywave',
  'mimic'],
 'poliwhirl': ['bide',
  'mega-kick',
  'double-slap',
  'amnesia',
  'psychic',
  'skull-bash',
  'substitute',
  'earthquake',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'hypnosis',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'double-team',
  'bubble',
  'double-edge',
  'mega-punch',
  'ice-beam',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'metronome'],
 'poliwrath': ['hyper-beam',
  'bide',
  'mega-kick',
  'double-slap',
  'amnesia',
  'psychic',
  'skull-bash',
  'substitute',
  'earthquake',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'hypnosis',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'double-team',
  'bubble',
  'double-edge',
  'mega-punch',
  'ice-beam',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'metronome'],
 'abra': ['tri-attack',
  'mega-kick',
  'teleport',
  'bide',
  'reflect',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'double-team',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'metronome'],
 'kadabra': ['tri-attack',
  'bide',
  'teleport',
  'mega-kick',
  'disable',
  'reflect',
  'psybeam',
  'recover',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'double-team',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'kinesis',
  'seismic-toss',
  'thunder-wave',
  'metronome'],
 'alakazam': ['tri-attack',
  'hyper-beam',
  'bide',
  'teleport',
  'mega-kick',
  'disable',
  'reflect',
  'psybeam',
  'recover',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'double-team',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'kinesis',
  'seismic-toss',
  'thunder-wave',
  'metronome'],
 'machop': ['mega-kick',
  'bide',
  'rock-slide',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'metronome',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'karate-chop',
  'strength',
  'leer',
  'double-team',
  'low-kick',
  'mega-punch',
  'double-edge',
  'body-slam',
  'mimic',
  'seismic-toss',
  'focus-energy'],
 'machoke': ['mega-kick',
  'bide',
  'rock-slide',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'focus-energy',
  'karate-chop',
  'take-down',
  'strength',
  'leer',
  'low-kick',
  'double-team',
  'mega-punch',
  'double-edge',
  'body-slam',
  'mimic',
  'seismic-toss',
  'metronome'],
 'machamp': ['hyper-beam',
  'mega-kick',
  'bide',
  'rock-slide',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'focus-energy',
  'karate-chop',
  'take-down',
  'strength',
  'leer',
  'low-kick',
  'double-team',
  'mega-punch',
  'double-edge',
  'body-slam',
  'mimic',
  'seismic-toss',
  'metronome'],
 'bellsprout': ['bide',
  'acid',
  'razor-leaf',
  'reflect',
  'mega-drain',
  'slam',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'sleep-powder',
  'rest',
  'growth',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'wrap',
  'double-edge',
  'cut',
  'poison-powder',
  'stun-spore',
  'mimic'],
 'weepinbell': ['bide',
  'acid',
  'razor-leaf',
  'reflect',
  'mega-drain',
  'slam',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'rest',
  'sleep-powder',
  'growth',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'wrap',
  'double-edge',
  'cut',
  'poison-powder',
  'stun-spore',
  'mimic'],
 'victreebel': ['hyper-beam',
  'bide',
  'acid',
  'razor-leaf',
  'reflect',
  'mega-drain',
  'slam',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'rest',
  'sleep-powder',
  'growth',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'wrap',
  'double-edge',
  'cut',
  'body-slam',
  'poison-powder',
  'stun-spore',
  'mimic'],
 'tentacool': ['constrict',
  'bide',
  'acid',
  'reflect',
  'screech',
  'barrier',
  'skull-bash',
  'mega-drain',
  'substitute',
  'swords-dance',
  'rest',
  'poison-sting',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'supersonic',
  'double-team',
  'wrap',
  'double-edge',
  'ice-beam',
  'cut',
  'mimic'],
 'tentacruel': ['constrict',
  'hyper-beam',
  'bide',
  'acid',
  'reflect',
  'screech',
  'barrier',
  'skull-bash',
  'mega-drain',
  'substitute',
  'swords-dance',
  'rest',
  'poison-sting',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'supersonic',
  'double-team',
  'wrap',
  'double-edge',
  'ice-beam',
  'cut',
  'mimic'],
 'geodude': ['explosion',
  'bide',
  'tackle',
  'rock-slide',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'self-destruct',
  'rage',
  'toxic',
  'rock-throw',
  'submission',
  'take-down',
  'strength',
  'double-team',
  'harden',
  'mega-punch',
  'double-edge',
  'defense-curl',
  'body-slam',
  'mimic',
  'seismic-toss',
  'metronome'],
 'graveler': ['explosion',
  'bide',
  'tackle',
  'rock-slide',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'self-destruct',
  'rage',
  'toxic',
  'rock-throw',
  'submission',
  'take-down',
  'strength',
  'double-team',
  'harden',
  'mega-punch',
  'double-edge',
  'defense-curl',
  'body-slam',
  'mimic',
  'seismic-toss',
  'metronome'],
 'golem': ['explosion',
  'hyper-beam',
  'bide',
  'mega-kick',
  'tackle',
  'rock-slide',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'self-destruct',
  'rage',
  'toxic',
  'rock-throw',
  'submission',
  'take-down',
  'strength',
  'double-team',
  'harden',
  'mega-punch',
  'double-edge',
  'defense-curl',
  'body-slam',
  'mimic',
  'seismic-toss',
  'metronome'],
 'ponyta': ['bide',
  'ember',
  'reflect',
  'skull-bash',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'tail-whip',
  'growl',
  'double-team',
  'horn-drill',
  'stomp',
  'double-edge',
  'swift',
  'body-slam',
  'fire-spin',
  'mimic'],
 'rapidash': ['hyper-beam',
  'bide',
  'ember',
  'reflect',
  'skull-bash',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'tail-whip',
  'growl',
  'double-team',
  'horn-drill',
  'stomp',
  'double-edge',
  'swift',
  'body-slam',
  'fire-spin',
  'mimic'],
 'slowpoke': ['tri-attack',
  'bide',
  'teleport',
  'disable',
  'reflect',
  'amnesia',
  'psychic',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'flash',
  'rest',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'headbutt',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'thunder-wave'],
 'slowbro': ['tri-attack',
  'hyper-beam',
  'bide',
  'teleport',
  'mega-kick',
  'disable',
  'reflect',
  'amnesia',
  'psychic',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'flash',
  'rest',
  'counter',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'headbutt',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'withdraw',
  'seismic-toss',
  'thunder-wave'],
 'magnemite': ['thunderbolt',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'screech',
  'substitute',
  'flash',
  'rest',
  'rage',
  'toxic',
  'thunder-shock',
  'take-down',
  'supersonic',
  'double-team',
  'sonic-boom',
  'double-edge',
  'swift',
  'mimic',
  'thunder-wave',
  'thunder'],
 'magneton': ['thunderbolt',
  'hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'screech',
  'substitute',
  'flash',
  'rest',
  'rage',
  'toxic',
  'thunder-shock',
  'take-down',
  'supersonic',
  'double-team',
  'sonic-boom',
  'double-edge',
  'swift',
  'mimic',
  'thunder-wave',
  'thunder'],
 'farfetchd': ['bide',
  'reflect',
  'skull-bash',
  'agility',
  'substitute',
  'whirlwind',
  'swords-dance',
  'fury-attack',
  'rest',
  'rage',
  'toxic',
  'slash',
  'sand-attack',
  'take-down',
  'leer',
  'double-team',
  'double-edge',
  'cut',
  'peck',
  'swift',
  'body-slam',
  'mimic',
  'razor-wind'],
 'doduo': ['tri-attack',
  'bide',
  'reflect',
  'skull-bash',
  'agility',
  'substitute',
  'whirlwind',
  'fury-attack',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'take-down',
  'drill-peck',
  'growl',
  'double-team',
  'double-edge',
  'peck',
  'body-slam',
  'mimic'],
 'dodrio': ['tri-attack',
  'hyper-beam',
  'bide',
  'reflect',
  'skull-bash',
  'agility',
  'substitute',
  'whirlwind',
  'fury-attack',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'take-down',
  'drill-peck',
  'growl',
  'double-team',
  'double-edge',
  'peck',
  'body-slam',
  'mimic'],
 'seel': ['bide',
  'skull-bash',
  'substitute',
  'rest',
  'pay-day',
  'surf',
  'rage',
  'aurora-beam',
  'bubble-beam',
  'water-gun',
  'toxic',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'horn-drill',
  'headbutt',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic'],
 'dewgong': ['hyper-beam',
  'bide',
  'skull-bash',
  'substitute',
  'rest',
  'pay-day',
  'surf',
  'rage',
  'aurora-beam',
  'bubble-beam',
  'water-gun',
  'toxic',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'horn-drill',
  'headbutt',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic'],
 'grimer': ['explosion',
  'thunderbolt',
  'poison-gas',
  'bide',
  'acid-armor',
  'disable',
  'screech',
  'mega-drain',
  'substitute',
  'fire-blast',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'pound',
  'sludge',
  'double-team',
  'harden',
  'body-slam',
  'mimic',
  'thunder'],
 'muk': ['explosion',
  'thunderbolt',
  'poison-gas',
  'hyper-beam',
  'bide',
  'acid-armor',
  'disable',
  'screech',
  'mega-drain',
  'substitute',
  'fire-blast',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'pound',
  'sludge',
  'double-team',
  'harden',
  'body-slam',
  'mimic',
  'thunder'],
 'shellder': ['explosion',
  'tri-attack',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'substitute',
  'rest',
  'self-destruct',
  'surf',
  'rage',
  'aurora-beam',
  'bubble-beam',
  'water-gun',
  'toxic',
  'clamp',
  'blizzard',
  'take-down',
  'supersonic',
  'leer',
  'double-team',
  'double-edge',
  'ice-beam',
  'swift',
  'mimic',
  'withdraw'],
 'cloyster': ['explosion',
  'tri-attack',
  'hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'substitute',
  'rest',
  'self-destruct',
  'surf',
  'clamp',
  'aurora-beam',
  'bubble-beam',
  'water-gun',
  'toxic',
  'rage',
  'blizzard',
  'take-down',
  'supersonic',
  'leer',
  'double-team',
  'spike-cannon',
  'double-edge',
  'ice-beam',
  'swift',
  'mimic',
  'withdraw'],
 'gastly': ['explosion',
  'thunderbolt',
  'confuse-ray',
  'bide',
  'psychic',
  'mega-drain',
  'substitute',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'hypnosis',
  'dream-eater',
  'double-team',
  'night-shade',
  'thunder',
  'psywave',
  'mimic',
  'lick'],
 'haunter': ['explosion',
  'thunderbolt',
  'confuse-ray',
  'bide',
  'psychic',
  'mega-drain',
  'substitute',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'hypnosis',
  'dream-eater',
  'double-team',
  'night-shade',
  'thunder',
  'psywave',
  'mimic',
  'lick'],
 'gengar': ['explosion',
  'thunderbolt',
  'confuse-ray',
  'hyper-beam',
  'bide',
  'mega-kick',
  'psychic',
  'skull-bash',
  'mega-drain',
  'substitute',
  'rest',
  'counter',
  'self-destruct',
  'rage',
  'toxic',
  'hypnosis',
  'submission',
  'take-down',
  'lick',
  'strength',
  'dream-eater',
  'double-team',
  'night-shade',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder',
  'metronome'],
 'onix': ['explosion',
  'bide',
  'tackle',
  'rock-slide',
  'screech',
  'skull-bash',
  'slam',
  'substitute',
  'earthquake',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'rock-throw',
  'take-down',
  'strength',
  'double-team',
  'harden',
  'double-edge',
  'bind',
  'body-slam',
  'mimic'],
 'drowzee': ['tri-attack',
  'poison-gas',
  'mega-kick',
  'teleport',
  'bide',
  'meditate',
  'disable',
  'reflect',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'hypnosis',
  'pound',
  'submission',
  'take-down',
  'dream-eater',
  'double-team',
  'headbutt',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'metronome'],
 'hypno': ['tri-attack',
  'poison-gas',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'meditate',
  'disable',
  'reflect',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'hypnosis',
  'pound',
  'submission',
  'take-down',
  'dream-eater',
  'double-team',
  'headbutt',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'metronome'],
 'krabby': ['bide',
  'crabhammer',
  'substitute',
  'swords-dance',
  'rest',
  'surf',
  'vice-grip',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'bubble',
  'stomp',
  'harden',
  'double-edge',
  'ice-beam',
  'cut',
  'body-slam',
  'mimic'],
 'kingler': ['hyper-beam',
  'bide',
  'crabhammer',
  'substitute',
  'swords-dance',
  'rest',
  'surf',
  'vice-grip',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'bubble',
  'stomp',
  'harden',
  'double-edge',
  'ice-beam',
  'cut',
  'body-slam',
  'mimic'],
 'voltorb': ['explosion',
  'thunderbolt',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'screech',
  'substitute',
  'flash',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'light-screen',
  'sonic-boom',
  'swift',
  'mimic',
  'thunder-wave',
  'thunder'],
 'electrode': ['explosion',
  'thunderbolt',
  'hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'screech',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'light-screen',
  'sonic-boom',
  'swift',
  'mimic',
  'thunder-wave',
  'thunder'],
 'exeggcute': ['explosion',
  'egg-bomb',
  'bide',
  'teleport',
  'reflect',
  'psychic',
  'substitute',
  'solar-beam',
  'sleep-powder',
  'rest',
  'self-destruct',
  'rage',
  'toxic',
  'hypnosis',
  'leech-seed',
  'take-down',
  'double-team',
  'double-edge',
  'psywave',
  'poison-powder',
  'stun-spore',
  'mimic',
  'barrage'],
 'exeggutor': ['explosion',
  'egg-bomb',
  'hyper-beam',
  'bide',
  'teleport',
  'reflect',
  'psychic',
  'mega-drain',
  'substitute',
  'solar-beam',
  'rest',
  'sleep-powder',
  'self-destruct',
  'leech-seed',
  'toxic',
  'hypnosis',
  'rage',
  'take-down',
  'strength',
  'double-team',
  'stomp',
  'double-edge',
  'psywave',
  'poison-powder',
  'stun-spore',
  'mimic',
  'barrage'],
 'cubone': ['thrash',
  'mega-kick',
  'bide',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'bonemerang',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'leer',
  'growl',
  'double-team',
  'headbutt',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'seismic-toss',
  'bone-club',
  'focus-energy'],
 'marowak': ['thrash',
  'hyper-beam',
  'bide',
  'mega-kick',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'bonemerang',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'leer',
  'growl',
  'double-team',
  'headbutt',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'seismic-toss',
  'bone-club',
  'focus-energy'],
 'hitmonlee': ['jump-kick',
  'mega-kick',
  'bide',
  'meditate',
  'double-kick',
  'skull-bash',
  'substitute',
  'rolling-kick',
  'metronome',
  'rest',
  'high-jump-kick',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'strength',
  'double-team',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'mimic',
  'seismic-toss',
  'focus-energy'],
 'hitmonchan': ['thunder-punch',
  'mega-kick',
  'bide',
  'skull-bash',
  'agility',
  'substitute',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'comet-punch',
  'take-down',
  'ice-punch',
  'strength',
  'double-team',
  'fire-punch',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'mimic',
  'seismic-toss',
  'metronome'],
 'lickitung': ['thunderbolt',
  'hyper-beam',
  'mega-kick',
  'bide',
  'disable',
  'screech',
  'skull-bash',
  'slam',
  'substitute',
  'swords-dance',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'supersonic',
  'double-team',
  'stomp',
  'wrap',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'cut',
  'defense-curl',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder'],
 'koffing': ['toxic',
  'haze',
  'sludge',
  'explosion',
  'smokescreen',
  'thunderbolt',
  'substitute',
  'smog',
  'fire-blast',
  'bide',
  'tackle',
  'mimic',
  'rest',
  'double-team',
  'self-destruct',
  'thunder',
  'rage'],
 'weezing': ['toxic',
  'haze',
  'sludge',
  'explosion',
  'smokescreen',
  'thunderbolt',
  'substitute',
  'smog',
  'hyper-beam',
  'fire-blast',
  'bide',
  'tackle',
  'mimic',
  'rest',
  'double-team',
  'self-destruct',
  'thunder',
  'rage'],
 'rhyhorn': ['thunderbolt',
  'bide',
  'rock-slide',
  'horn-attack',
  'skull-bash',
  'substitute',
  'fire-blast',
  'fury-attack',
  'earthquake',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'strength',
  'tail-whip',
  'leer',
  'double-team',
  'horn-drill',
  'stomp',
  'double-edge',
  'body-slam',
  'mimic',
  'thunder'],
 'rhydon': ['thunderbolt',
  'hyper-beam',
  'bide',
  'mega-kick',
  'rock-slide',
  'horn-attack',
  'skull-bash',
  'substitute',
  'fire-blast',
  'fury-attack',
  'earthquake',
  'rest',
  'counter',
  'pay-day',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'leer',
  'double-team',
  'horn-drill',
  'stomp',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder'],
 'chansey': ['thunderbolt',
  'egg-bomb',
  'tri-attack',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'sing',
  'reflect',
  'double-slap',
  'psychic',
  'skull-bash',
  'substitute',
  'solar-beam',
  'fire-blast',
  'flash',
  'rest',
  'soft-boiled',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'pound',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'growl',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'defense-curl',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder',
  'metronome'],
 'tangela': ['absorb',
  'hyper-beam',
  'constrict',
  'bide',
  'skull-bash',
  'mega-drain',
  'slam',
  'substitute',
  'swords-dance',
  'solar-beam',
  'vine-whip',
  'sleep-powder',
  'rest',
  'growth',
  'rage',
  'toxic',
  'take-down',
  'double-team',
  'double-edge',
  'bind',
  'cut',
  'body-slam',
  'poison-powder',
  'stun-spore',
  'mimic'],
 'kangaskhan': ['thunderbolt',
  'hyper-beam',
  'mega-kick',
  'bide',
  'rock-slide',
  'skull-bash',
  'dizzy-punch',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'comet-punch',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'leer',
  'double-team',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'seismic-toss',
  'thunder'],
 'horsea': ['bide',
  'skull-bash',
  'agility',
  'substitute',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'bubble',
  'double-edge',
  'ice-beam',
  'smokescreen',
  'swift',
  'mimic'],
 'seadra': ['hyper-beam',
  'bide',
  'skull-bash',
  'substitute',
  'agility',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'bubble',
  'double-edge',
  'ice-beam',
  'smokescreen',
  'swift',
  'mimic'],
 'goldeen': ['bide',
  'horn-attack',
  'skull-bash',
  'agility',
  'substitute',
  'fury-attack',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'supersonic',
  'tail-whip',
  'double-team',
  'horn-drill',
  'double-edge',
  'ice-beam',
  'swift',
  'peck',
  'mimic',
  'waterfall'],
 'seaking': ['hyper-beam',
  'bide',
  'horn-attack',
  'skull-bash',
  'substitute',
  'agility',
  'fury-attack',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'supersonic',
  'tail-whip',
  'double-team',
  'horn-drill',
  'double-edge',
  'ice-beam',
  'swift',
  'peck',
  'mimic',
  'waterfall'],
 'staryu': ['thunderbolt',
  'tri-attack',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'recover',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'double-team',
  'light-screen',
  'harden',
  'double-edge',
  'ice-beam',
  'swift',
  'psywave',
  'mimic',
  'thunder-wave',
  'thunder'],
 'starmie': ['thunderbolt',
  'tri-attack',
  'hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'recover',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'double-team',
  'light-screen',
  'harden',
  'double-edge',
  'ice-beam',
  'swift',
  'psywave',
  'mimic',
  'thunder-wave',
  'thunder'],
 'mr-mime': ['thunderbolt',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'meditate',
  'reflect',
  'double-slap',
  'psychic',
  'barrier',
  'skull-bash',
  'substitute',
  'solar-beam',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'body-slam',
  'psywave',
  'confusion',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder',
  'metronome'],
 'scyther': ['wing-attack',
  'hyper-beam',
  'bide',
  'skull-bash',
  'agility',
  'substitute',
  'swords-dance',
  'rest',
  'rage',
  'toxic',
  'slash',
  'take-down',
  'leer',
  'double-team',
  'double-edge',
  'cut',
  'swift',
  'quick-attack',
  'mimic',
  'focus-energy'],
 'jynx': ['thrash',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'lovely-kiss',
  'reflect',
  'double-slap',
  'psychic',
  'skull-bash',
  'substitute',
  'rest',
  'counter',
  'rage',
  'toxic',
  'bubble-beam',
  'pound',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'ice-punch',
  'double-team',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'lick',
  'metronome'],
 'electabuzz': ['thunder-punch',
  'thunderbolt',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'reflect',
  'screech',
  'psychic',
  'skull-bash',
  'substitute',
  'flash',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'thunder-shock',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'light-screen',
  'mega-punch',
  'double-edge',
  'swift',
  'body-slam',
  'psywave',
  'quick-attack',
  'mimic',
  'seismic-toss',
  'thunder-wave',
  'thunder',
  'metronome'],
 'magmar': ['confuse-ray',
  'hyper-beam',
  'mega-kick',
  'teleport',
  'bide',
  'ember',
  'flamethrower',
  'psychic',
  'skull-bash',
  'substitute',
  'fire-blast',
  'rest',
  'counter',
  'rage',
  'toxic',
  'submission',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'fire-punch',
  'mega-punch',
  'double-edge',
  'smokescreen',
  'smog',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'metronome'],
 'pinsir': ['hyper-beam',
  'bide',
  'substitute',
  'swords-dance',
  'rest',
  'vice-grip',
  'rage',
  'toxic',
  'submission',
  'slash',
  'take-down',
  'strength',
  'double-team',
  'harden',
  'double-edge',
  'bind',
  'cut',
  'body-slam',
  'mimic',
  'seismic-toss',
  'focus-energy'],
 'tauros': ['thunderbolt',
  'hyper-beam',
  'bide',
  'tackle',
  'skull-bash',
  'substitute',
  'fire-blast',
  'earthquake',
  'rest',
  'rage',
  'toxic',
  'blizzard',
  'take-down',
  'strength',
  'tail-whip',
  'leer',
  'double-team',
  'horn-drill',
  'stomp',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'thunder'],
 'magikarp': ['splash', 'tackle'],
 'gyarados': ['thunderbolt',
  'dragon-rage',
  'hyper-beam',
  'bide',
  'tackle',
  'reflect',
  'skull-bash',
  'substitute',
  'fire-blast',
  'splash',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'double-edge',
  'ice-beam',
  'bite',
  'body-slam',
  'mimic',
  'thunder'],
 'lapras': ['thunderbolt',
  'dragon-rage',
  'confuse-ray',
  'hyper-beam',
  'bide',
  'sing',
  'reflect',
  'psychic',
  'skull-bash',
  'substitute',
  'mist',
  'solar-beam',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'strength',
  'growl',
  'double-team',
  'horn-drill',
  'double-edge',
  'ice-beam',
  'body-slam',
  'psywave',
  'mimic',
  'thunder'],
 'ditto': ['transform'],
 'eevee': ['bide',
  'tackle',
  'reflect',
  'skull-bash',
  'substitute',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'sand-attack',
  'tail-whip',
  'growl',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'quick-attack',
  'mimic',
  'focus-energy'],
 'vaporeon': ['haze',
  'hyper-beam',
  'bide',
  'acid-armor',
  'tackle',
  'reflect',
  'skull-bash',
  'substitute',
  'mist',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'aurora-beam',
  'blizzard',
  'take-down',
  'sand-attack',
  'tail-whip',
  'growl',
  'double-team',
  'double-edge',
  'ice-beam',
  'swift',
  'bite',
  'body-slam',
  'quick-attack',
  'mimic',
  'focus-energy'],
 'jolteon': ['thunderbolt',
  'hyper-beam',
  'bide',
  'tackle',
  'reflect',
  'double-kick',
  'skull-bash',
  'substitute',
  'agility',
  'flash',
  'rest',
  'rage',
  'toxic',
  'thunder-shock',
  'take-down',
  'sand-attack',
  'tail-whip',
  'growl',
  'double-team',
  'thunder',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'quick-attack',
  'mimic',
  'thunder-wave',
  'pin-missile',
  'focus-energy'],
 'flareon': ['hyper-beam',
  'bide',
  'tackle',
  'ember',
  'reflect',
  'flamethrower',
  'skull-bash',
  'substitute',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'take-down',
  'sand-attack',
  'tail-whip',
  'leer',
  'growl',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'body-slam',
  'smog',
  'quick-attack',
  'fire-spin',
  'mimic',
  'focus-energy'],
 'porygon': ['thunderbolt',
  'tri-attack',
  'hyper-beam',
  'bide',
  'teleport',
  'tackle',
  'reflect',
  'recover',
  'psybeam',
  'psychic',
  'skull-bash',
  'agility',
  'substitute',
  'flash',
  'rest',
  'sharpen',
  'rage',
  'toxic',
  'conversion',
  'blizzard',
  'take-down',
  'double-team',
  'double-edge',
  'ice-beam',
  'swift',
  'psywave',
  'mimic',
  'thunder-wave',
  'thunder'],
 'omanyte': ['bide',
  'reflect',
  'horn-attack',
  'substitute',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'spike-cannon',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'withdraw'],
 'omastar': ['hyper-beam',
  'bide',
  'reflect',
  'horn-attack',
  'skull-bash',
  'substitute',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'horn-drill',
  'spike-cannon',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic',
  'withdraw',
  'seismic-toss'],
 'kabuto': ['absorb',
  'bide',
  'reflect',
  'substitute',
  'scratch',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'slash',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'harden',
  'double-edge',
  'ice-beam',
  'body-slam',
  'mimic'],
 'kabutops': ['absorb',
  'hyper-beam',
  'bide',
  'mega-kick',
  'reflect',
  'skull-bash',
  'substitute',
  'scratch',
  'swords-dance',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'hydro-pump',
  'submission',
  'slash',
  'blizzard',
  'take-down',
  'seismic-toss',
  'leer',
  'double-team',
  'harden',
  'double-edge',
  'ice-beam',
  'cut',
  'body-slam',
  'mimic',
  'razor-wind'],
 'aerodactyl': ['dragon-rage',
  'wing-attack',
  'hyper-beam',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'whirlwind',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'take-down',
  'supersonic',
  'double-team',
  'double-edge',
  'swift',
  'bite',
  'mimic',
  'razor-wind'],
 'snorlax': ['thunderbolt',
  'hyper-beam',
  'mega-kick',
  'bide',
  'reflect',
  'rock-slide',
  'amnesia',
  'psychic',
  'skull-bash',
  'substitute',
  'solar-beam',
  'fire-blast',
  'earthquake',
  'rest',
  'counter',
  'pay-day',
  'surf',
  'self-destruct',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'submission',
  'blizzard',
  'take-down',
  'strength',
  'double-team',
  'headbutt',
  'harden',
  'mega-punch',
  'double-edge',
  'ice-beam',
  'body-slam',
  'psywave',
  'mimic',
  'seismic-toss',
  'thunder',
  'metronome'],
 'articuno': ['hyper-beam',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'mist',
  'whirlwind',
  'rest',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'sky-attack',
  'blizzard',
  'take-down',
  'double-team',
  'double-edge',
  'ice-beam',
  'swift',
  'peck',
  'mimic',
  'razor-wind'],
 'zapdos': ['thunderbolt',
  'hyper-beam',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'whirlwind',
  'flash',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'thunder-shock',
  'take-down',
  'drill-peck',
  'double-team',
  'light-screen',
  'double-edge',
  'swift',
  'mimic',
  'razor-wind',
  'thunder-wave',
  'thunder'],
 'moltres': ['hyper-beam',
  'bide',
  'reflect',
  'agility',
  'substitute',
  'whirlwind',
  'fire-blast',
  'rest',
  'rage',
  'toxic',
  'sky-attack',
  'take-down',
  'leer',
  'double-team',
  'double-edge',
  'swift',
  'peck',
  'fire-spin',
  'mimic',
  'razor-wind'],
 'dratini': ['thunderbolt',
  'dragon-rage',
  'hyper-beam',
  'bide',
  'reflect',
  'skull-bash',
  'slam',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'wrap',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'mimic',
  'thunder-wave',
  'thunder'],
 'dragonair': ['thunderbolt',
  'dragon-rage',
  'hyper-beam',
  'bide',
  'reflect',
  'skull-bash',
  'slam',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'leer',
  'double-team',
  'horn-drill',
  'wrap',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'mimic',
  'thunder-wave',
  'thunder'],
 'dragonite': ['thunderbolt',
  'dragon-rage',
  'hyper-beam',
  'bide',
  'reflect',
  'skull-bash',
  'slam',
  'agility',
  'substitute',
  'fire-blast',
  'rest',
  'surf',
  'rage',
  'toxic',
  'bubble-beam',
  'water-gun',
  'blizzard',
  'take-down',
  'strength',
  'leer',
  'double-team',
  'horn-drill',
  'wrap',
  'double-edge',
  'ice-beam',
  'swift',
  'body-slam',
  'mimic',
  'razor-wind',
  'thunder-wave',
  'thunder']}

In [17]:
#dict of which types moves are physical and which are special
phys_spec_dict = {'Normal':'Physical','Fighting':'Physical','Flying':'Physical','Poison':'Physical',
                  'Ground':'Physical','Rock':'Physical','Bug':'Physical','Ghost':'Physical','Fire':'Special',
                  'Water':'Special','Grass':'Special','Psychic':'Special','Ice':'Special','Dragon':'Special'}

In [18]:
boost_dict = {-6:0.25,-5:0.28,-4:0.33,-3:0.4,-2:0.5,-1:0.66,0:1,1:1.5,2:2,3:2.5,4:3,5:3.5,6:4}

In [19]:

crit_dict = {
    "electrode": 27.34,
    "aerodactyl": 25.39,
    "jolteon": 25.39,
    "mewtwo": 25.39,
    "alakazam": 23.44,
    "dugtrio": 23.44,
    "persian": 22.46,
    "starmie": 22.46,
    "gengar": 21.48,
    "tauros": 21.48,
    "electabuzz": 20.50,
    "rapidash": 20.50,
    "scyther": 20.50,
    "charizard": 19.53,
    "dodrio": 19.53,
    "fearow": 19.53,
    "mew": 19.53,
    "ninetales": 19.53,
    "raichu": 19.53,
    "tentacruel": 19.53,
    "zapdos": 19.53,
    "raticate": 18.95,
    "arcanine": 18.55,
    "jynx": 18.55,
    "primeape": 18.55,
    "magmar": 18.16,
    "pidgeot": 17.77,
    "golbat": 17.59,
    "kangaskhan": 17.59,
    "moltres": 17.59,
    "mr. mime": 17.59,
    "venomoth": 17.59,
    "hitmonlee": 16.99,
    "articuno": 16.60,
    "golduck": 16.60,
    "nidoking": 16.60,
    "pinsir": 16.60,
    "seadra": 16.60,
    "gyarados": 15.82,
    "arbok": 15.63,
    "dragonite": 15.63,
    "kabutops": 15.63,
    "venusaur": 15.63,
    "blastoise": 15.23,
    "hitmonchan": 14.84,
    "nidoqueen": 14.84,
    "beedrill": 14.65,
    "kingler": 14.65,
    "butterfree": 13.67,
    "cloyster": 13.67,
    "dewgong": 13.67,
    "magneton": 13.67,
    "onix": 13.67,
    "poliwrath": 13.67,
    "victreebel": 13.67,
    "seaking": 13.28,
    "hypno": 13.09,
    "flareon": 12.70,
    "sandslash": 12.70,
    "vaporeon": 12.70,
    "clefable": 11.71,
    "farfetch'd": 11.71,
    "lapras": 11.71,
    "tangela": 11.71,
    "weezing": 11.71,
    "exeggutor": 10.74,
    "machamp": 10.74,
    "omastar": 10.74,
    "chansey": 9.77,
    "muk": 9.77,
    "vileplume": 9.77,
    "ditto": 9.38,
    "golem": 8.79,
    "marowak": 8.79,
    "wigglytuff": 8.79,
    "porygon": 7.81,
    "rhydon": 7.81,
    "lickitung": 5.86,
    "parasect": 5.86,
    "slowbro": 5.86,
    "snorlax": 5.86
}
for k in crit_dict:
    crit_dict[k]/=100


In [20]:
tb = TeambuilderPokemon(species='snorlax',moves=['BodySlam','Earthquake','Rest','Reflect'],evs=[252]*6,ivs=[15]*15,level=100)

In [21]:
lax = Pokemon(gen=1,species='snorlax',teambuilder = tb)

In [22]:
physical = MoveCategory(1)
special = MoveCategory(2)
status_glob = MoveCategory(3)
BRN = Status(1)
FNT = Status(2)
FRZ = Status(3)
PAR = Status(4)
PSN = Status(5)
SLP = Status(6)
TOX = Status(7)

In [23]:
import math
import bisect
import copy

In [24]:
move = Move('bodyslam',1)

In [25]:
move.expected_hits

1

In [26]:
def ko_chance(hp,dmg):
    """
    function to compute odds of ko
    hp - target estimated hp
    dmg - damage amounts 
    
    """
    idx = bisect.bisect_right(dmg, hp)
    
    return (39-idx)/39

ko_chance(10,[127, 128, 128, 129, 130, 130, 131, 131, 132, 132, 133, 134, 134, 135, 135, 136, 137, 137, 138, 138, 139, 140, 140, 141, 141, 142, 142, 143, 144, 144, 145, 145, 146, 147, 147, 148, 148, 149, 150])

1.0

In [ ]:
class bot_v1_q(Player):


    def get_state(self, battle: AbstractBattle):
        player_team = battle.team
        opponent_team = battle.opponent_team
        player_mon = battle.active_pokemon
        opponent_mon = battle.opponent_active_pokemon
        return (player_team,player_mon,opponent_team,opponent_mon)
    
    def hp_score(self,hp):
        """
        simple function to assign a score to hp
        hp - health as a decimal
        """
        
        score = 2.5 + (7.5)*hp
        return score

    def score(self,player_team,opponent_team):



        active_mon = None
        opp_active_mon = None
        curr_score = 0
        opp_score = 0
        for k in player_team:
            mon = player_team[k]

            if mon.active:
                active_mon = mon
            else:
                hp = mon.current_hp_fraction
                score = self.hp_score(hp)
                status = mon.status
                if status == FNT:
                    score = 0
                elif status == FRZ:
                    score*=(1/20)
                elif status == PAR:
                    score*=(1/2)
                curr_score+=score


        
        for k in opponent_team:
            
            mon = opponent_team[k]

            if mon.active:
                opp_active_mon = mon
            else:
                hp = mon.current_hp_fraction
                score = self.hp_score(hp)
                status = mon.status
                if status == FNT:
                    score = 0
                elif status == FRZ:
                    score*=(1/20)
                elif status == PAR:
                    score*=(1/2)
                opp_score+=score
        active_mon_score = self.hp_score(active_mon.current_hp_fraction)
        act_status = active_mon.status
        if act_status == FRZ:
            active_mon_score*=(1/20)
        elif act_status == PAR:
            active_mon_score*=(1/2)
        elif act_status == FNT:
            active_mon_score = 0
    

        curr_score+=active_mon_score
        opp_mon_score = self.hp_score(opp_active_mon.current_hp_fraction)
        opp_status = opp_active_mon.status
        if opp_status == FRZ:
            opp_mon_score*=(1/20)
        elif opp_status == PAR:
            opp_mon_score*=(1/2)
        elif opp_status == FNT:
            opp_mon_score = 0
        opp_score+=opp_mon_score
        opp_score+=(10*(2 - len(opponent_team)))
        if curr_score == 0:
            return -100
        elif opp_score ==0:
            return 100


        return curr_score-opp_score
    
    def ko_chance(self,hp,dmg):
        """
        function to compute odds of ko
        hp - target estimated hp
        dmg - damage amounts 
        
        """
        idx = bisect.bisect_right(dmg, hp)
        return (39-idx)/39
    
    def create_alt_team(self,orig_team,orig_mon,new_mon):
        new_team = {}
        for k in orig_team:
            if orig_team[k] == orig_mon:
                new_team[k] = new_mon
            else:
                new_team[k] = orig_team[k]
        return new_team

    def predict(self,curr_team,curr_mon,opp_team,opp_mon,player_move,opponent_move):


        opp_mon_name = opp_mon.species
        curr_mon_name = curr_mon.species
        

        opp_boosts = opp_mon.boosts
        curr_boosts = curr_mon.boosts
        opp_status = opp_mon.status
        curr_status = curr_mon.status
        if curr_status:
            curr_hp_status = str(curr_mon.current_hp)+'/'+str(curr_mon.max_hp) + curr_status.name
        else:
            curr_hp_status = str(curr_mon.current_hp)+'/'+str(curr_mon.max_hp)
        if opp_status:
            opp_hp_status = str(opp_mon.current_hp)+'/'+str(opp_mon.max_hp) + opp_status.name
        else:
            opp_hp_status = str(opp_mon.current_hp)+'/'+str(opp_mon.max_hp)
        curr_speed = stats.loc[curr_mon_name]['Speed_Total']
        opp_speed = stats.loc[opp_mon_name]['Speed_Total']
        curr_speed = math.floor(curr_speed*(boost_dict[curr_boosts['spe']]))
        opp_speed = math.floor(opp_speed*(boost_dict[opp_boosts['spe']]))
        opp_copy = copy.deepcopy(opp_mon)
        stats_dict = {'atk': stats.loc[opp_mon_name]['Attack_Total'] , 'def': stats.loc[opp_mon_name]['Defense_Total'] , 'hp': stats.loc[opp_mon_name]['HP_Total'],
                      'spa': stats.loc[opp_mon_name]['Special_Total'] , 'spd': stats.loc[opp_mon_name]['Special_Total'] , 'spe': stats.loc[opp_mon_name]['Speed_Total'] }
        for key in opp_copy.stats:
            opp_copy.stats[key] = stats_dict[key]



        player_copy = copy.deepcopy(curr_mon)



        if curr_status == 'PAR':
            curr_speed = curr_speed//4  
        if opp_status == 'PAR':
            opp_speed = opp_speed//4

        first_move = 1
        if curr_speed>opp_speed:
            first_move = 2
        elif opp_speed>curr_speed:
            first_move = 3

        curr_stab = 1
        if curr_mon.type_1 == player_move.type or curr_mon.type_2 == player_move.type:
            curr_stab = 1.5
        player_move_type = player_move.type
        curr_type_mult = player_move_type.damage_multiplier(opp_mon.type_1,type_chart = TypeChart)
        if opp_mon.type_2:
            curr_type_mult*=player_move_type.damage_multiplier(opp_mon.type_2,type_chart = TypeChart)

        opp_stab = 1
        if opp_mon.type_1 == opponent_move.type or opp_mon.type_2 == opponent_move.type:
            opp_stab = 1.5
        opponent_move_type = opponent_move.type
        opp_type_mult = opponent_move_type.damage_multiplier(curr_mon.type_1,type_chart = TypeChart)
        if curr_mon.type_2:
            opp_type_mult*=opponent_move_type.damage_multiplier(curr_mon.type_2,type_chart = TypeChart)
        if not player_move: #player move is None, player is switching but variables still need to be set
            dmg_dealt = None
            dmg_dealt_crit = None
            dmg_dealt_smpl = None
            dmg_dealt_crit_smpl = None

        elif player_move.category == physical:


            dmg_dealt = damage(100,False,player_move.base_power,stats.loc[curr_mon_name]['Attack_Total'],stats.loc[opp_mon_name]['Defense_Total'],boost_dict[curr_boosts['atk']],boost_dict[opp_boosts['def']],curr_stab,curr_type_mult)
            dmg_dealt_crit = damage(100,True,player_move.base_power,stats.loc[curr_mon_name]['Attack_Total'],stats.loc[opp_mon_name]['Defense_Total'],boost_dict[curr_boosts['atk']],boost_dict[opp_boosts['def']],curr_stab,curr_type_mult)
            dmg_dealt_smpl = [dmg_dealt[0], dmg_dealt[19], dmg_dealt[-1]]
            dmg_dealt_crit_smpl = [dmg_dealt_crit[0], dmg_dealt_crit[19], dmg_dealt_crit[-1]]
        elif player_move.category == special:

            dmg_dealt = damage(100,False,player_move.base_power,stats.loc[curr_mon_name]['Special_Total'],stats.loc[opp_mon_name]['Special_Total'],boost_dict[curr_boosts['spa']],boost_dict[opp_boosts['spd']],curr_stab,curr_type_mult)
            dmg_dealt_crit = damage(100,True,player_move.base_power,stats.loc[curr_mon_name]['Special_Total'],stats.loc[opp_mon_name]['Special_Total'],boost_dict[curr_boosts['spa']],boost_dict[opp_boosts['spd']],curr_stab,curr_type_mult)                    
            dmg_dealt_smpl = [dmg_dealt[0], dmg_dealt[19], dmg_dealt[-1]]
            dmg_dealt_crit_smpl = [dmg_dealt_crit[0], dmg_dealt_crit[19], dmg_dealt_crit[-1]]
        curr_crit_chance = crit_dict[curr_mon_name]
        normal_ko_chance =  self.ko_chance(int(opp_copy.current_hp*opp_copy.max_hp),dmg_dealt)
        crit_ko_chance = self.ko_chance(int(opp_copy.current_hp*opp_copy.max_hp),dmg_dealt_crit)
        curr_ko_chance = (1-curr_crit_chance)*normal_ko_chance + curr_crit_chance*crit_ko_chance
        if not opponent_move: #opponent move is None, opponent is switching but variables still need to be set
            dmg_rcv = None
            dmg_rcv_crit = None
            dmg_rcv_smpl = None
            dmg_rcv_crit_smpl = None

        elif opponent_move.category == physical:


            dmg_rcv = damage(100,False,player_move.base_power,stats.loc[opp_mon_name]['Attack_Total'],stats.loc[curr_mon_name]['Defense_Total'],boost_dict[opp_boosts['atk']],boost_dict[opp_boosts['def']],opp_stab,opp_type_mult)
            dmg_rcv_crit = damage(100,True,player_move.base_power,stats.loc[opp_mon_name]['Attack_Total'],stats.loc[curr_mon_name]['Defense_Total'],boost_dict[opp_boosts['atk']],boost_dict[opp_boosts['def']],opp_stab,opp_type_mult)
            dmg_rcv_smpl = [dmg_rcv[0], dmg_rcv[19], dmg_rcv[-1]]
            dmg_rcv_crit_smpl = [dmg_rcv_crit[0], dmg_rcv_crit[19], dmg_rcv_crit[-1]]
        elif player_move.category == special:

            dmg_rcv = damage(100,False,opponent_move.base_power,stats.loc[opp_mon_name]['Special_Total'],stats.loc[curr_mon_name]['Special_Total'],boost_dict[opp_boosts['spa']],boost_dict[opp_boosts['spd']],opp_stab,opp_type_mult)
            dmg_rcv_crit = damage(100,True,opponent_move.base_power,stats.loc[opp_mon_name]['Special_Total'],stats.loc[curr_mon_name]['Special_Total'],boost_dict[opp_boosts['spa']],boost_dict[opp_boosts['spd']],opp_stab,opp_type_mult)                    
            dmg_rcv_smpl = [dmg_rcv[0], dmg_rcv[19], dmg_rcv[-1]]
            dmg_rcv_crit_smpl = [dmg_rcv_crit[0], dmg_rcv_crit[19], dmg_rcv_crit[-1]]
        opp_crit_chance = crit_dict[opp_mon_name]
        opp_normal_ko_chance =  self.ko_chance(player_copy.current_hp,dmg_rcv)
        opp_crit_ko_chance = self.ko_chance(player_copy.current_hp,dmg_rcv_crit)
        opp_ko_chance = (1-opp_crit_chance)*opp_normal_ko_chance + opp_crit_chance*opp_crit_ko_chance

        curr_score = 0

        for v,k in zip(dmg_dealt_smpl,dmg_rcv_smpl):
            player_copy.set_hp_status(curr_hp_status)
            opp_copy.set_hp_status(opp_hp_status)

            new_player_hp = int(max(player_copy.current_hp - k,0))
            player_status_name = player_copy.status.name if player_copy.status else ''
            player_copy.set_hp_status(str(new_player_hp)+'/'+str(player_copy.max_hp) + player_status_name)

            new_player_team = self.create_alt_team(orig_team = curr_team,orig_mon=curr_mon,new_mon=player_copy)
            new_opponent_hp = int(max((1/100)*opp_copy.current_hp*(opp_copy.stats['hp']) - v,0))

            opp_status_name = opp_copy.status.name if opp_copy.status else ''

            opp_copy.set_hp_status(str(new_opponent_hp)+'/'+str(opp_copy.stats['hp']) + opp_status_name)

            new_opponent_team = self.create_alt_team(orig_team = opp_team,orig_mon=opp_mon,new_mon=opp_copy)

            curr_score += 1/3*(self.score(new_player_team,new_opponent_team))
        return curr_score

    def choose_move(self, battle):
        state = self.get_state(battle)
        print()
        x = self.score(state[0],state[2])
        y = self.predict(state[0],state[1],state[2],state[3],move,move)

        return self.choose_random_move(battle)

In [ ]:
team_1 = """
Cloyster  
Ability: No Ability  
- Clamp
- Body Slam
"""
team_2 = """
Cloyster
Ability: No Ability
- Clamp  
- Body Slam
"""

In [29]:
from poke_env import RandomPlayer

In [30]:
p1 = RandomPlayer(battle_format='gen1ou',team = team_1)
p2 = bot_v1_q(battle_format='gen1ou',team = team_2)

In [ ]:
await p1.battle_against(p2, n_battles=1)

2025-02-05 18:01:39,748 - bot_v1_q 1 - ERROR - Unhandled exception raised while handling message:
>battle-gen1ou-14277
|init|battle
|title|RandomPlayer 1 vs. bot_v1_q 1
|j|☆RandomPlayer 1
|j|☆bot_v1_q 1
|t:|1738799207
|gametype|singles
|player|p1|RandomPlayer 1|1|
|player|p2|bot_v1_q 1|170|
|teamsize|p1|1
|teamsize|p2|1
|gen|1
|tier|[Gen 1] OU
|rule|Desync Clause Mod: Desyncs changed to move failure.
|rule|Sleep Clause Mod: Limit one foe put to sleep
|rule|Freeze Clause Mod: Limit one foe frozen
|rule|Species Clause: Limit one of each Pokémon
|rule|OHKO Clause: OHKO moves are banned
|rule|Evasion Moves Clause: Evasion moves are banned
|rule|Endless Battle Clause: Forcing endless battles is banned
|rule|HP Percentage Mod: HP is shown in percentages
|
|t:|1738799207
|start
|switch|p1a: Snorlax|Snorlax|100/100
|switch|p2a: Nidoran-F|Nidoran-F|313/313
|turn|1
|l|☆bot_v1_q 1
|player|p2|
|l|☆RandomPlayer 1
|player|p1|
|j|☆RandomPlayer 1
|player|p1|RandomPlayer 1|1|
|j|☆bot_v1_q 1
|player|p2|